In [1]:
print("hello")

hello


In [10]:
import os
import pandas as pd
from glob import glob

base_skin_dir = "/Users/csengeszoke/programming/Thesis/Code/Fairness-Rectification/Data"

imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_skin_dir, '*', '*.jpg'))}

lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

metadata_path = os.path.join(base_skin_dir, 'HAM10000_metadata.csv')

tile_df = pd.read_csv(metadata_path)

tile_df['path'] = tile_df['image_id'].map(imageid_path_dict.get)
tile_df['cell_type'] = tile_df['dx'].map(lesion_type_dict.get) 
tile_df['cell_type_idx'] = pd.Categorical(tile_df['cell_type']).codes

tile_df.sample(3)

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,cell_type,cell_type_idx
5589,HAM_0000056,ISIC_0025725,nv,follow_up,55.0,female,trunk,/Users/csengeszoke/programming/Thesis/Code/Fai...,Melanocytic nevi,4
606,HAM_0005865,ISIC_0031522,bkl,histo,70.0,male,chest,/Users/csengeszoke/programming/Thesis/Code/Fai...,Benign keratosis-like lesions,2
5039,HAM_0005793,ISIC_0024629,nv,follow_up,65.0,male,trunk,/Users/csengeszoke/programming/Thesis/Code/Fai...,Melanocytic nevi,4


In [11]:
df = pd.DataFrame()

df['filepaths'] =tile_df['path']
df['labels'] =tile_df['cell_type_idx']
df['labels'] =df['labels'].apply(lambda x: str(x))
df.head()

,filepaths,labels
0,/Users/csengeszoke/programming/Thesis/Code/Fai...,2
1,/Users/csengeszoke/programming/Thesis/Code/Fai...,2
2,/Users/csengeszoke/programming/Thesis/Code/Fai...,2
3,/Users/csengeszoke/programming/Thesis/Code/Fai...,2
4,/Users/csengeszoke/programming/Thesis/Code/Fai...,2


In [12]:
# This df will be used for fairness evaluation
df2 = pd.DataFrame() 

df2['filepaths'] = tile_df['path']
df2['labels']    = tile_df['cell_type_idx']
#df2['labels']    = df['labels'].apply(lambda x: str(x))
df2['age']       = tile_df['age']
df2['sex']       = tile_df['sex']
df2.head()

gender_counts = df2['sex'].value_counts()
print(gender_counts/len(df2))

age_counts = df2['age'].value_counts()
print(age_counts)

sex
male       0.539790
female     0.454518
unknown    0.005691
Name: count, dtype: float64
age
45.0    1299
50.0    1187
55.0    1009
40.0     985
60.0     803
70.0     756
35.0     753
65.0     731
75.0     618
30.0     464
80.0     404
85.0     290
25.0     247
20.0     169
5.0       86
15.0      77
10.0      41
0.0       39
Name: count, dtype: int64
